# Create Typical Meteorological Year wind and solar profiles

- Take the wind and solar files from 1990-2019 for each region (30 years)
- Average each 8760 hour across all years
- Drop leap days from this averaging
- This creates a TMY profile based on 30 years of data
- Re-expand this to 30 years of data in the same format as the wind and solar CFs files
- Copy values from Feb 28 to any leap day Feb 29

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import calendar

sys.path.append("/home/truggles/Inter-annual_Variability_Residual_Load")
from helpers import return_file_info_map

In [ ]:
def get_resource_file(ba, resource):
    ba_info = return_file_info_map( ba )
    df = pd.read_csv(f"../{ba_info[resource][0]}")
    print(df.head())
    return df

def drop_leap_days(df):
    df = df.loc[ ~((df['month'] == 2) & (df['day'] == 29)) ]
    return df

def calc_TMY_profile(df, years):
    
    var = 'w_cfs' if 'w_cfs' in df.columns else 's_cfs'
    tmy = np.zeros(8760) # hours in non-leap year
    for yr in years:
        tmp = df.loc[ df['year'] == yr ]
        tmy += tmp[var].values
    tmy = tmy / len(years)
    return tmy

def distribute_TMY(ba, resource, tmy, years):
    df = get_resource_file(ba, resource)
    df['tmy'] = np.zeros(len(df.index))
    for yr in years:
        df.loc[ (df['year'] == yr) & ~((df['month'] == 2) & (df['day'] == 29)), 'tmy' ] = tmy
        if calendar.isleap(yr):
            df.loc[ (df['year'] == yr) & (df['month'] == 2) & (df['day'] == 29), 'tmy' ] = \
               df.loc[ (df['year'] == yr) & (df['month'] == 2) & (df['day'] == 28), 'tmy' ].values 
    ba_info = return_file_info_map( ba )
    
    # Drop normal CF column
    var = 'w_cfs' if 'w_cfs' in df.columns else 's_cfs'
    df = df.drop([var,], axis=1)
    df = df.rename(columns = {'tmy':var})
    
    df.to_csv(f"../{ba_info[resource][0]}".replace('.csv', '_TMY.csv'), index=False)

def multi_week_avgs(ba, resource, shifts_min, shifts_max, years):
    
    ba_info = return_file_info_map( ba )
    df = pd.read_csv(f"../{ba_info[resource][0]}".replace('.csv', '_TMY.csv'))
    var = 'w_cfs' if 'w_cfs' in df.columns else 's_cfs'
    df[f'{var}2'] = np.zeros(len(df.index))
    n_shifts = 0
    for i in range(shifts_min, shifts_max+1):
        print(i, 24*i)
        df[f'{var}2'] += df[var].shift(periods=24*i)
        n_shifts += 1
    df[f'{var}2'] = df[f'{var}2'] / n_shifts
    
    # Fix ends where shifts returned NAN values
    for yr in [years[0], years[-1]]:
        print(yr)
        if calendar.isleap(yr):
            print(f'Is leap {yr}')
            tgt_yr = yr + 4 if yr + 4 in years else yr - 4
        if not calendar.isleap(yr):
            print(f'Is not leap {yr}')
            tgt_yr = yr + 1 if yr + 1 in years else yr - 1
        print(tgt_yr)
        df.loc[ df['year'] == yr, f'{var}2' ] = df.loc[ df['year'] == tgt_yr, f'{var}2' ].values
    
    # Drop normal CF column
    to_save = df
    to_save = to_save.drop([var,], axis=1)
    to_save = to_save.rename(columns = {f'{var}2':var})
    to_save.to_csv(f"../{ba_info[resource][0]}".replace('.csv', '_TMY2.csv'), index=False)
    return df
    
years = range(1990, 2020)
for resource in ['wind', 'solar']:
    for ba in ['ERCOT', 'PJM', 'NYISO', 'FR']:
        df = get_resource_file(ba, resource)
        df = drop_leap_days(df)
        tmy = calc_TMY_profile(df, years)
        distribute_TMY(ba, resource, tmy, years)
        df = multi_week_avgs(ba, resource, -7, 7, years)

In [ ]:
fig, ax = plt.subplots(figsize=(15,3))
#ax.plot(tmy)
ax.plot(df.loc[df['year']==2019, 'w_cfs'].values)
ax.plot(df.loc[df['year']==2019, 'w_cfs2'].values, alpha=.5)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,3))
#ax.plot(tmy)
ax.plot(df.loc[(df['year']==2019) & (df['month']==7), 'w_cfs'].values)
ax.plot(df.loc[(df['year']==2019) & (df['month']==7), 'w_cfs2'].values, alpha=.5)
plt.show()